# Introduction

The dam-level data can be downloaded in csv format from the City of Cape Town's Open Data Portal [here](https://web1.capetown.gov.za/web1/opendataportal/DatasetDetail?DatasetName=Dam+levels). The file isn't formatted in the most ready-to-analyse way. In this script, I wrangle the data and write the output to a 'clean' file that I use to analyse the data in subsequent scripts.

There are a few things that I end up doing to the data:
+ Deal with column names. In their original form, they are not amedable to easily referencing the data
+ Deal with incorrect dates
+ Convert the data from *wide* to *long* format

In [11]:
import numpy as np
import pandas as pd
import re
import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

First lets open the csv file outside of python (which you can do in spreadsheet software, such as Excel or Google Sheets, or in a text editor). Doing this, we can see that the 1st row is a title, which is useless to us, and the next 3 lines are column headers. Knowing this helps us to figure out how to read the data into our DataFrame.

Now, let's read the data and see what we're dealing with. We can see from the outset that we're going to need to (1) process the column names, and (2) change the format of the dataset from 'wide' to 'long' format so that it can be easily referenced.

In [2]:
file_location = 'data/Dam-levels-update-2012-2017_20171212.csv'
data = pd.read_csv(file_location, header = [2,3,4], encoding = 'latin-1') # Rows 2, 3, and 4 are column headers
data.head()

,Unnamed: 0_level_0,WEMMERSHOEK,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,STEENBRAS LOWER,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,STEENBRAS UPPER,...,Unnamed: 51_level_0,Unnamed: 52_level_0,TOTAL STORED - BIG 6,Unnamed: 54_level_0,Unnamed: 55_level_0,Unnamed: 56_level_0,LAND-en ZEEZICHT,Unnamed: 58_level_0,Unnamed: 59_level_0,Unnamed: 60_level_0
,DATE,HEIGHT,STORAGE,Current,Last Year,HEIGHT,STORAGE,Current,Last Year,HEIGHT,...,Current,Last Year,STORAGE,Current,Last Year,Unnamed: 56_level_1,HEIGHT,STORAGE,Current,Last Year
,Unnamed: 0_level_2,(m),(Ml),%,%,(m),(Ml),%,%,(m),...,%,%,(Ml),%,%,Unnamed: 56_level_2,(m),(Ml),%,%
0,01-Jan-12,48.23,44 621,76.1,NaN,20.34,23 549,70.3,NaN,24.17,...,89.2,NaN,695 783,77.46,NaN,NaN,NaN,NaN,NaN,NaN
1,02-Jan-12,48.21,44 571,76.0,NaN,20.31,23 460,70.0,NaN,24.17,...,88.3,NaN,692 990,77.15,NaN,NaN,NaN,NaN,NaN,NaN
2,03-Jan-12,48.17,44 471,75.8,NaN,20.28,23 372,69.7,NaN,24.15,...,87.9,NaN,690 324,76.85,NaN,NaN,NaN,NaN,NaN,NaN
3,04-Jan-12,48.13,44 372,75.7,NaN,20.26,23 313,69.6,NaN,24.17,...,87.5,NaN,687 282,76.52,NaN,NaN,NaN,NaN,NaN,NaN
4,05-Jan-12,48.11,44 322,75.6,NaN,20.23,23 224,69.3,NaN,24.05,...,87.2,NaN,683 355,76.08,NaN,NaN,NaN,NaN,NaN,NaN


# Collapsing the column names

First let's deal with the column headers and 'flatten' them out into a single level. At the moment, it's not easy to reference the data. For instance, if I want the 'height' for the Wemmershoek dam, I have to do quite a lot of unnecessary work. It would be easier if the name of the dam and the variable name were in the same header 'level'. This will also help us to convert the data from it's current 'wide' format to 'long', which we'll do soon.

The columns names have been represented in the DataFrame in a MultiIndex, which allows them to have multiple levels. Where it says 'Unnamed...', this represents a blank space in the original csv file.

We can also see that totals have been pre-calculated for the Big 6 dams. We don't want this is the final dataset we'll remove them.

In [3]:
data.columns

MultiIndex(levels=[['ALEXANDRA', 'BERG RIVER', 'DE VILLIERS', 'HELY-HUTCHINSON', 'KLEINPLAATS', 'LAND-en ZEEZICHT', 'LEWIS GAY', 'STEENBRAS LOWER', 'STEENBRAS UPPER', 'THEEWATERSKLOOF', 'TOTAL STORED - BIG 6', 'Unnamed: 0_level_0', 'Unnamed: 10_level_0', 'Unnamed: 11_level_0', 'Unnamed: 12_level_0', 'Unnamed: 14_level_0', 'Unnamed: 15_level_0', 'Unnamed: 16_level_0', 'Unnamed: 18_level_0', 'Unnamed: 19_level_0', 'Unnamed: 20_level_0', 'Unnamed: 22_level_0', 'Unnamed: 23_level_0', 'Unnamed: 24_level_0', 'Unnamed: 26_level_0', 'Unnamed: 27_level_0', 'Unnamed: 28_level_0', 'Unnamed: 2_level_0', 'Unnamed: 30_level_0', 'Unnamed: 31_level_0', 'Unnamed: 32_level_0', 'Unnamed: 34_level_0', 'Unnamed: 35_level_0', 'Unnamed: 36_level_0', 'Unnamed: 38_level_0', 'Unnamed: 39_level_0', 'Unnamed: 3_level_0', 'Unnamed: 40_level_0', 'Unnamed: 42_level_0', 'Unnamed: 43_level_0', 'Unnamed: 44_level_0', 'Unnamed: 46_level_0', 'Unnamed: 47_level_0', 'Unnamed: 48_level_0', 'Unnamed: 4_level_0', 'Unnamed: 50

In [4]:
# Break multi-level column names into individual lists
header_0 = [h[0] for h in data.columns[1:]]
header_1 = [h[1] for h in data.columns[1:]]
header_2 = [h[2] for h in data.columns[1:]]

# Fill 'Unnamed' top-level column names with dam names from left to right
for idx, h in enumerate(header_0):
    if re.search(r'^Unnamed', h) is not None:
        header_0[idx] = header_0[idx - 1]

header_0 = [h.title() for h in header_0]
header_1 = [h.lower() for h in header_1]
header_2 = [h.lower() for h in header_2]

col_names = ['date']
# Join all 3 levels of the column names together
col_names = col_names + list(map('_'.join, zip(header_0, header_1, header_2)))

data.columns = pd.Index(col_names)
# Get rid of 'totals'
data = data.drop([col for col in data.columns if re.match(r'^Total Stored', col)], axis = 1)

data['date'] = pd.to_datetime(data['date'])

In [5]:
data.head()

,date,Wemmershoek_height_(m),Wemmershoek_storage_(ml),Wemmershoek_current_%,Wemmershoek_last year_%,Steenbras Lower_height_(m),Steenbras Lower_storage_(ml),Steenbras Lower_current_%,Steenbras Lower_last year_%,Steenbras Upper_height_(m),...,Theewaterskloof_current_%,Theewaterskloof_last year_%,Berg River_height_(m),Berg River_storage_(ml),Berg River_current_%,Berg River_last year_%,Land-En Zeezicht_height_(m),Land-En Zeezicht_storage_(ml),Land-En Zeezicht_current_%,Land-En Zeezicht_last year_%
0,2012-01-01,48.23,44 621,76.1,NaN,20.34,23 549,70.3,NaN,24.17,...,74.5,NaN,39.33,115 930,89.2,NaN,NaN,NaN,NaN,NaN
1,2012-01-02,48.21,44 571,76.0,NaN,20.31,23 460,70.0,NaN,24.17,...,74.3,NaN,39.12,114 850,88.3,NaN,NaN,NaN,NaN,NaN
2,2012-01-03,48.17,44 471,75.8,NaN,20.28,23 372,69.7,NaN,24.15,...,74.0,NaN,39.01,114 280,87.9,NaN,NaN,NaN,NaN,NaN
3,2012-01-04,48.13,44 372,75.7,NaN,20.26,23 313,69.6,NaN,24.17,...,73.7,NaN,38.91,113 770,87.5,NaN,NaN,NaN,NaN,NaN
4,2012-01-05,48.11,44 322,75.6,NaN,20.23,23 224,69.3,NaN,24.05,...,73.1,NaN,38.84,113 410,87.2,NaN,NaN,NaN,NaN,NaN


# Dealing with dates

It's always a good idea to sense-check data. Dates are especially susceptible to being messed up because of the various different ways that they can be formatted.

We can see below that the minimum date is 1 Jan 2012. This is fine. However, the maximum date is 22 May *2021*, which is obviously wrong, since that date hasn't yet occured. We are going to need to investigate what's going on here a little further.

In [6]:
data.shape
data['date'].min()
data['date'].max()

(2182, 57)

Timestamp('2012-01-01 00:00:00')

Timestamp('2021-05-22 00:00:00')

Looking at the last 20 rows of the dataset, we can see that the last date is really '2017-12-06', even though there are a few more rows after that with dates but no data.

In [7]:
data.tail(20)

,date,Wemmershoek_height_(m),Wemmershoek_storage_(ml),Wemmershoek_current_%,Wemmershoek_last year_%,Steenbras Lower_height_(m),Steenbras Lower_storage_(ml),Steenbras Lower_current_%,Steenbras Lower_last year_%,Steenbras Upper_height_(m),...,Theewaterskloof_current_%,Theewaterskloof_last year_%,Berg River_height_(m),Berg River_storage_(ml),Berg River_current_%,Berg River_last year_%,Land-En Zeezicht_height_(m),Land-En Zeezicht_storage_(ml),Land-En Zeezicht_current_%,Land-En Zeezicht_last year_%
2162,2017-12-02,41.68,30110,51.343701,51.613123,18.45,18334,54.700600,48.942328,25.01,...,22.918940,46.031971,32.92,84600,65.071918,66.833321,8.0,473.149,104.9110865,NaN
2163,2017-12-03,41.70,30149,51.410204,51.309597,18.39,18182,54.247098,48.366501,25.01,...,22.804610,46.031971,32.87,84370,64.895008,66.617952,8.0,473.149,104.9110865,NaN
2164,2017-12-04,41.73,30208,51.510811,50.975377,18.38,18157,54.172509,48.151684,25.02,...,22.539714,46.031971,32.85,84280,64.825783,66.333359,8.0,473.149,104.9110865,NaN
2165,2017-12-05,41.74,30228,51.544915,50.637746,18.38,18157,54.172509,47.936868,25.03,...,22.352079,45.392013,32.80,84060,64.656565,66.479502,8.0,473.149,104.9110865,NaN
2166,2017-12-06,41.75,30248,51.579019,50.274538,18.34,18056,53.871170,47.289435,24.94,...,22.165693,45.264980,32.75,83830,64.479655,65.771864,8.0,473.149,104.9110865,NaN
2167,2017-12-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2168,2017-12-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2169,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2170,2017-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2171,2017-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since the last few rows aren't at all useful, I'll get rid of them

In [8]:
data = data[:2167]

Now let's see what's up with the dates. To see this, we can follow this process:
+ 1) Determine the index of the 'bad' dates (greater than '2017-12-12)
+ 2) Find where these 'bad' dates begin and end
+ 3) Print dates just before the problem starts, problematic dates, and dates just after the problem ends - in that order
+ 4) Figure out what could be causing the 'bad' dates to occur by looking at the problematic dates in the context of the correct dates

In [9]:
bad_date_idx = data.loc[data['date'] > '2017-12-12'].index
max_bad_idx = max(bad_date_idx)
min_bad_idx = min(bad_date_idx)

print('The dates seem fine here...')
data.iloc[min_bad_idx-20:min_bad_idx]['date'].tolist()  
print('...The problem seems to start here...')
data.iloc[min_bad_idx:max_bad_idx+1]['date'].tolist()
print('...The problem seems to end there. Let\'s see the next few rows...')
data.loc[max_bad_idx+1:max_bad_idx+10]['date'].tolist()

The dates seem fine here...


[Timestamp('2017-04-29 00:00:00'),
 Timestamp('2017-04-30 00:00:00'),
 Timestamp('2017-01-05 00:00:00'),
 Timestamp('2017-02-05 00:00:00'),
 Timestamp('2017-03-05 00:00:00'),
 Timestamp('2017-04-05 00:00:00'),
 Timestamp('2017-05-05 00:00:00'),
 Timestamp('2017-06-05 00:00:00'),
 Timestamp('2017-07-05 00:00:00'),
 Timestamp('2017-08-08 00:00:00'),
 Timestamp('2017-09-05 00:00:00'),
 Timestamp('2017-10-05 00:00:00'),
 Timestamp('2017-11-05 00:00:00'),
 Timestamp('2017-12-05 00:00:00'),
 Timestamp('2017-05-13 00:00:00'),
 Timestamp('2017-05-14 00:00:00'),
 Timestamp('2017-05-15 00:00:00'),
 Timestamp('2017-05-16 00:00:00'),
 Timestamp('2017-05-17 00:00:00'),
 Timestamp('2017-05-18 00:00:00')]

...The problem seems to start here...


[Timestamp('2018-05-19 00:00:00'),
 Timestamp('2019-05-20 00:00:00'),
 Timestamp('2020-05-21 00:00:00'),
 Timestamp('2021-05-22 00:00:00')]

...The problem seems to end there. Let's see the next few rows...


[Timestamp('2017-05-23 00:00:00'),
 Timestamp('2017-05-24 00:00:00'),
 Timestamp('2017-05-25 00:00:00'),
 Timestamp('2017-05-26 00:00:00'),
 Timestamp('2017-05-27 00:00:00'),
 Timestamp('2017-05-28 00:00:00'),
 Timestamp('2017-05-29 00:00:00'),
 Timestamp('2017-05-30 00:00:00'),
 Timestamp('2017-05-31 00:00:00'),
 Timestamp('2017-06-01 00:00:00')]

We can see that what has happened is that the *year* part of the date from '2017-05-18' to '2017-05-22' has been incremented. This probably happened in spreadsheet software, such as Excel, where the person inputting the dates used autofill somewhere along the line, which increments numbers automatically. So all we need to do is fix up the year part of those dates.

In order to keep this general, though, let's just fill in all the dates in the data set from start to finish. We might want to read updates of the dam-levels data in at a later date and automate this cleaning. Filling the dates will get rid of any date-related issues that may arise in the future (assuming that no dates are ever skipped in future datasets). We can see that no dates are skipped in *this* dataset because the difference in days between the minimum and maximum dates in the dataset is equal the number of rows.

In [12]:
(datetime.datetime(year = 2017, month = 12, day = 6) - data['date'].min()).days + 1
data.shape[0]

2167

2167

In [13]:
for idx, d in data['date'].iteritems():
    if idx > 0:
        data.iloc[idx, 0] = data.iloc[idx - 1, 0] + datetime.timedelta(days = 1)

In [14]:
data['date'].min()
data['date'].max()

Timestamp('2012-01-01 00:00:00')

Timestamp('2017-12-06 00:00:00')

# It's getting hot in here: melting our DataFrame

The data is 'wide' formatted (also known as 'pivoted'), with columns names repeated horizontally for each dam. This isn't very easy for computers to read, so we'll need to do some wrangling to transform this into 'long' format. When we're done, there will be no duplicate column names but there will be an additional column to identify each dam. Essentially, instead of having the data for each dam next to each other, we'll have them underneath each other.

In [15]:
# Get distinct list of dam names
dam_names = set(header_0)
dam_names = [d.title() for d in dam_names if not d.startswith('Total Stored')]

cols = data.columns.tolist()[1:]
dates = data['date']

dam_levels = pd.DataFrame()
for dam in dam_names:
    next_dam = pd.DataFrame({'date': dates,
                            'dam_name': dam})
    for col in cols:
        # If the dam name is in the column name, we process the data and associate it with that dam
        if re.search(dam, col):
            measure = re.sub(dam+'_', '', col) # Get rid of dam name from header (leaves us with the measure name)
            next_dam[measure] = data[col].replace(' ', '', regex = True).replace('#VALUE!', np.nan, regex = True)
            next_dam[measure] = next_dam[measure].astype(float)    
    dam_levels = pd.concat([dam_levels, next_dam], axis = 0)

dam_names = [dam.lower() for dam in dam_names]

In [16]:
dam_levels = dam_levels.rename(columns = {'height_(m)': 'height_m', 
                            'storage_(ml)': 'storage_ml',
                            'last year (%)': 'lastyear_%'})

# Writing to disk

And that's it, we're done! The final thing to do is write this all to a csv file.

In [19]:
dam_levels.head()

,dam_name,date,height_m,storage_ml,current_%,last year_%
0,Victoria,2012-01-01,0.0,0.0,0.0,NaN
1,Victoria,2012-01-02,0.0,0.0,0.0,NaN
2,Victoria,2012-01-03,0.0,0.0,0.0,NaN
3,Victoria,2012-01-04,0.0,0.0,0.0,NaN
4,Victoria,2012-01-05,0.0,0.0,0.0,NaN


In [20]:
dam_levels.to_csv('data/Dam-levels-clean-20120101-20171206.csv',
                 index = False)